In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import csv
import nltk
from matplotlib import pyplot as plt
import os
import re
import warnings
import random
warnings.filterwarnings('ignore')
from collections import Counter,defaultdict
from nltk import word_tokenize
from math import log
import string
import pickle
from math import log

import gensim
from gensim import corpora
from gensim import models
from string import digits
from collections import defaultdict
import ast
cwd = os.getcwd()

In [2]:
osname = os.name
if osname =='Windows':
    sym = "//"
else:
    sym = "/"

In [3]:
r_path=cwd+sym+'nips_reviewer_data'+sym+'reviewers.txt'
review_df=pd.read_csv(r_path,sep='\t',header=None)
review_df.columns=['sno','name']
review_lt=list(review_df.name)
review_sno={val:ind for ind,val in enumerate(review_lt)}

In [4]:
data_df=pd.read_pickle('data_df.pkl')

In [6]:
#paper_local.to_pickle('paper_net.pkl')
paper_net=pd.read_pickle('paper_net.pkl')

In [7]:
path1=cwd+sym+'Saved_Items'+sym
with open(path1+'paper_vec.pkl','rb') as f:
    paper_vec=pickle.load(f)
with open(path1+'paper_id.pkl','rb') as f:
    paper_id=pickle.load(f)

### Training

In [10]:
id2sn={}
for sn,pid in enumerate(paper_id):
    id2sn[pid]=sn

In [11]:
paper_vec2={}
for pid in paper_id:
    samp_df=data_df[data_df['id']==pid]
    index_lt=samp_df.index
    op=[]
    for ind in index_lt:
        op.append(paper_vec[ind])
    paper_vec2[id2sn[pid]]=np.array((sum(op)/len(op)))
    

In [13]:
len(paper_vec2[0])

250

In [41]:
paper_auth={}
for pid in paper_id:
    samp_df=data_df[data_df['id']==pid]
    index_lt=samp_df.index
    op=[]
    for ind in index_lt:
        op.append(review_sno[samp_df['author'][ind]])
    paper_auth[id2sn[pid]]=op

In [44]:
'''path1=cwd+sym+'Saved_Items'+sym
with open(path1+'paper_auth.pkl','wb') as f:
    pickle.dump(paper_auth,f)'''

In [12]:
'''path1=cwd+sym+'Saved_Items'+sym
with open(path1+'paper_vec2.pkl','wb') as f:
    pickle.dump(paper_vec2,f)'''

In [13]:
'''path1=cwd+sym+'Saved_Items'+sym
with open(path1+'paper_id.pkl','wb') as f:
    pickle.dump(paper_id,f)'''

### Training

In [23]:
adj_list={}
for ind,pid in enumerate(paper_id):
    samp_df=paper_net[paper_net['src']==pid]
    n=len(samp_df)
    outlink=[]
    for i in range(n):
        dest=samp_df['dest'].iloc[i]
        if dest in paper_id:
            outlink.append(id2sn[dest])
    if len(outlink):
        adj_list[ind]=outlink
    


In [10]:
'''with open('adj_list.pkl','wb') as f:
    pickle.dump(adj_list,f)'''

### Loading

In [25]:
with open('adj_list.pkl','rb') as f:
    adj_list=pickle.load(f)

In [26]:
from igraph import * 
g=Graph(directed=True)
n=len(paper_id)
g.add_vertices(n)
keys=list(adj_list.keys())
edge_lt=[]
for key in keys:
    for ele in adj_list[key]:
        #print((key,ele))
        edge_lt.append((key,ele))
        
    

In [27]:
g.add_edges(edge_lt)

In [28]:
bc=g.bibcoupling()

In [29]:
m=len(bc)
a=[sum(bc[i])for i in range(m)]

In [30]:
cocit=g.cocitation()

In [31]:
m=len(cocit)
b=[sum(cocit[i])for i in range(m)]

In [32]:
g.to_undirected()
dist=g.shortest_paths_dijkstra()

In [33]:
cocit_mat=b
bc_mat=a

In [25]:
'''path1=cwd+sym+'Saved_Items'+sym
with open(path1+'cocit.pkl','wb') as f:
    pickle.dump(cocit_mat,f)
with open(path1+'bibcoupling.pkl','wb') as f:
    pickle.dump(bc_mat,f)
with open(path1+'distance.pkl','wb') as f:
    pickle.dump(dist,f)'''

"'with open(path1+'distance.pkl','wb') as f:\n    pickle.dump(dist,f)"